# Visualização de Dados em Python


Neste notebook, vamos praticar a questão de divisão de classes com o método de Otsu. 

Se for necessário acrescentar passos adicionais além daqueles que estão indicados neste notebook, faça isso.


## Imagem de Entrada

A imagem de entrada "cameraman.jpg" está na página da semana no AVA.

O código a seguir faz a leitura da imagem e apresenta a imagem lida.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

image = plt.imread('cameraman.jpg') 

image.shape


O shape é interessante para saber o tamanho da matriz. 

O código a seguir apenas mostra a imagem lida. Confira o tamanho e a localização do ponto (0,0).


In [ ]:
plt.imshow(image, cmap='gray' )
plt.show()


A função abaixo implementa o limiar de Otsu. 


In [ ]:
def threshold_otsu(image, nbins=256):
    """Return threshold value based on Otsu's method.
    Parameters
    ----------
    image : (N, M) ndarray
        Grayscale input image.
    nbins : int, optional
        Number of bins used to calculate histogram. This value is ignored for
        integer arrays.
    Returns
    -------
    threshold : float
        Upper threshold value. All pixels with an intensity higher than
        this value are assumed to be foreground.
    Raises
    ------
    ValueError
         If `image` only contains a single grayscale value.
    References
    ----------
    .. [1] Wikipedia, https://en.wikipedia.org/wiki/Otsu's_Method
    Examples
    --------
    >>> from skimage.data import camera
    >>> image = camera()
    >>> thresh = threshold_otsu(image)
    >>> binary = image <= thresh
    Notes
    -----
    The input image must be grayscale.
    """
    if len(image.shape) > 2 and image.shape[-1] in (3, 4):
        msg = "threshold_otsu is expected to work correctly only for " \
              "grayscale images; image shape {0} looks like an RGB image"
        print(msg.format(image.shape))

    # Check if the image is multi-colored or not
    if image.min() == image.max():
        raise ValueError("threshold_otsu is expected to work with images "
                         "having more than one color. The input image seems "
                         "to have just one color {0}.".format(image.min()))

    hi, bin_centers, z = plt.hist(image.ravel(), nbins)
    hi = hi.astype(float)

    # class probabilities for all possible thresholds
    weight1 = np.cumsum(hi)
    weight2 = np.cumsum(hi[::-1])[::-1]
    # class means for all possible thresholds
    mean1 = np.cumsum(hi * bin_centers[0:256]) / weight1
    mean2 = (np.cumsum((hi * bin_centers[0:256])[::-1]) / weight2[::-1])[::-1]

    # Clip ends to align class 1 and class 2 variables:
    # The last value of `weight1`/`mean1` should pair with zero values in
    # `weight2`/`mean2`, which do not exist.
    variance12 = weight1[:-1] * weight2[1:] * (mean1[:-1] - mean2[1:]) ** 2

    idx = np.argmax(variance12)
    threshold = bin_centers[:-1][idx]
    return threshold

O código a seguir chama a função de Otsu e cria uma imagem Black & White. 

Verifique qual foi o valor do limiar.


In [ ]:
thresh = threshold_otsu(image)
binary = image > thresh
thresh

O código abaixo serve para mostrar a imagem original, o histograma com a marcação do limiar e a imagem resultante. 


In [ ]:
plt.figure(figsize=(8, 2.5))
plt.subplot(1, 3, 1)
plt.imshow(image, cmap='gray')
plt.title('Original')
plt.axis('off')

plt.subplot(1, 3, 2, aspect='equal')
plt.hist(image)
plt.title('Histogram')
plt.axvline(thresh, color='r')

plt.subplot(1, 3, 3)
plt.imshow(binary, cmap='gray')
plt.title('Thresholded')
plt.axis('off')

plt.show()